# Setup

In [4]:
import os
import openai
from openai import OpenAI

openai.api_key = os.environ["OPENAI_API_KEY"]

client = OpenAI()


def llm(prompt, stop=["\n"]):
    chat_completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        # messages=[{"role": "user", "content": prompt}, {"role": "system", "content": "You are a serious system and deep thinker, do not return the answer ambiguous answered the question, do not mix the meaning of found and acquire a company."}],
        messages=[{"role": "system", "content": prompt}],
        temperature=0,
        max_tokens=100,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=stop
    )
    last_message = chat_completion.choices[0].message.content
    return last_message

# deprecated code:
# def llm(prompt, stop=["\n"]):
#     response = openai.Completion.create(
#       model="text-davinci-002",
#       prompt=prompt,
#       temperature=0,
#       max_tokens=100,
#       top_p=1,
#       frequency_penalty=0.0,
#       presence_penalty=0.0,
#       stop=stop
#     )
#     return response["choices"][0]["text"]

In [5]:
import wikienv, wrappers
env = wikienv.WikiEnv()
env = wrappers.HotPotQAWrapper(env, split="dev")
env = wrappers.LoggingWrapper(env)

def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1

In [6]:
#naive_poc

import json
import sys

folder = './prompts/'
prompt_file = 'prompts_naive.json'
with open(folder + prompt_file, 'r') as f:
    prompt_dict = json.load(f)

webthink_examples = prompt_dict['webthink_simple6']
instruction = """Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be four types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
(3) Check[answer] Before providing the answer, please carefully review and verify the accuracy of every term and expression in the original question. If the answer is ambiguous or does not fully address the question, please return more similar entities to search.
(4) Finish[answer], which returns the answer and finishes the task.
Here are some examples.
"""
webthink_prompt = instruction + webthink_examples

print(webthink_prompt)

def webthinkPoc(question, prompt=webthink_prompt, to_print=True):
    # question = env.reset(idx=idx)
    if to_print:
        print("Poc question", question)
    prompt += question + "\n"
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):
        n_calls += 1
        thought_action = llm(prompt + f"Thought {i}:", stop=[f"\nObservation {i}:"])
        print('thought_action', thought_action)
        try:
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
        except:
            print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            action = llm(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
        print("action output start")
        print(action[0].lower())
        print(action[1:])
        print("action output end")
        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        if to_print:
            print(step_str)
        if done:
            break
    if not done:
        obs, r, done, info = step(env, "finish[]")
    if to_print:
        print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    return r, info

question = 'When was the  Argentine former footballer which  Dutch manager Ronald Koeman replaced in  2014-15 Southampton F.C. season born'
# question = 'What company did Rex Maughan aquire?'
r, info = webthinkPoc(question, to_print=True)

Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be four types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
(3) Check[answer] Before providing the answer, please carefully review and verify the accuracy of every term and expression in the original question. If the answer is ambiguous or does not fully address the question, please return more similar entities to search.
(4) Finish[answer], which returns the answer and finishes the task.
Here are some examples.

Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado or